In [ ]:
import random
import json

# 1. 문제 유형 pool 정의
problems = [
    {
        "problem": "1부터 10까지 출력하는 코드를 작성하세요.",
        "buggy_code": "for i in range(1, 10):\n    print(i)",
        "error_reason": "range의 끝값이 10이면 9까지만 출력됩니다.",
        "correct_code": "for i in range(1, 11):\n    print(i)",
        "input_case": "없음",
        "expected_output": "1\n2\n3\n4\n5\n6\n7\n8\n9\n10"
    },
    {
        "problem": "리스트의 평균을 계산하는 프로그램을 작성하세요.",
        "buggy_code": "nums = input().split(',')\nprint(sum(nums)/len(nums))",
        "error_reason": "입력값은 문자열이므로 sum이 작동하지 않습니다.",
        "correct_code": "nums = list(map(int, input().split(',')))\nprint(sum(nums)/len(nums))",
        "input_case": "10,20,30",
        "expected_output": "20.0"
    },
    {
        "problem": "입력한 정수가 짝수인지 홀수인지 출력하세요.",
        "buggy_code": "num = input()\nif num % 2 == 0:\n    print('짝수')\nelse:\n    print('홀수')",
        "error_reason": "input()은 문자열로 반환되므로 int 변환이 필요합니다.",
        "correct_code": "num = int(input())\nif num % 2 == 0:\n    print('짝수')\nelse:\n    print('홀수')",
        "input_case": "3",
        "expected_output": "홀수"
    },
    {
        "problem": "리스트의 짝수 값만 출력하세요.",
        "buggy_code": "nums = [1, 2, 3, 4, 5]\nfor n in nums:\n    if n % 2:\n        print(n)",
        "error_reason": "짝수가 아닌 값을 출력하고 있습니다.",
        "correct_code": "nums = [1, 2, 3, 4, 5]\nfor n in nums:\n    if n % 2 == 0:\n        print(n)",
        "input_case": "없음",
        "expected_output": "2\n4"
    },
    {
        "problem": "0부터 4까지의 제곱을 출력하세요.",
        "buggy_code": "for i in range(5):\n    print(i^2)",
        "error_reason": "`^`는 XOR 연산자입니다. 제곱은 `**`입니다.",
        "correct_code": "for i in range(5):\n    print(i**2)",
        "input_case": "없음",
        "expected_output": "0\n1\n4\n9\n16"
    },
]

# 2. 질문 템플릿 정의
question_templates = [
    "문제: {problem}\n코드:\n{buggy_code}\n에러:\n{error_reason}\n입력:\n{input_case}\n기대 출력:\n{expected_output}",
    "입력값이 {input_case}일 때, 아래 코드에서 에러가 발생했습니다:\n{buggy_code}\n에러 메시지:\n{error_reason}",
    "아래는 내가 작성한 코드야. 원하는 출력은 {expected_output}인데 잘 안 돼:\n{buggy_code}\n무슨 문제일까?"
]

# 3. 예제 생성 및 질문 포함
examples = []
for i in range(1, 101):
    base = random.choice(problems)
    question = random.choice(question_templates).format(
        problem=base["problem"],
        buggy_code=base["buggy_code"],
        error_reason=base["error_reason"],
        input_case=base["input_case"],
        expected_output=base["expected_output"]
    )
    examples.append({
        "id": i,
        "problem": base["problem"],
        "buggy_code": base["buggy_code"],
        "error_reason": base["error_reason"],
        "correct_code": base["correct_code"],
        "input_case": base["input_case"],
        "expected_output": base["expected_output"],
        "question": question
    })

# 4. 저장
output_path = "/mnt/data/dpo_examples_100_with_questions.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(examples, f, indent=2, ensure_ascii=False)

print(f"✅ 저장 완료: {output_path}")


In [ ]:
!pip install python-dotenv

In [ ]:
import openai
import os
import json
from dotenv import load_dotenv
from tqdm import tqdm

# 🔐 환경변수에서 API 키 로드
load_dotenv()
openai.api_key = ""

# ✅ 외부 JSON 파일에서 examples 불러오기
with open("/content/dpo_examples_100_with_questions.json", "r", encoding="utf-8") as f:
    examples = json.load(f)

# ❌ 오답 생성 프롬프트
def generate_incorrect(problem, buggy_code, question):
    prompt = f"""
문제:
{problem}

학생 질문:
{question}

학생이 작성한 코드:
{buggy_code}

요구사항:
- 질문에 대해 '엉뚱한 이유'로 틀렸다고 답변하세요.
- 실제 오류 원인을 절대 말하지 마세요.
- 코드의 다른 부분(들여쓰기, 변수명 등)을 문제라고 주장하세요.
- 틀린 설명을 자신 있게 제공하세요.
"""
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 고의로 잘못된 설명을 제공하는 AI 조교야."},
            {"role": "user", "content": prompt.strip()}
        ],
        temperature=0.9,
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

# ✅ 정답 생성 프롬프트
def generate_correct(problem, question, buggy_code, error_reason, correct_code, incorrect_answer):
    prompt = f"""
문제:
{problem}

질문:
{question}

학생이 작성한 코드:
{buggy_code}

학생이 틀린 이유:
{error_reason}

정답 코드:
{correct_code}

이전에 제공된 잘못된 답변:
{incorrect_answer}

요구사항:
- 정확한 원인을 설명하고, 정답 코드 기준으로 수정하세요.
- 초보자도 이해할 수 있도록 친절한 언어로 설명하세요.
"""
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 초보자를 위한 친절한 파이썬 조교야."},
            {"role": "user", "content": prompt.strip()}
        ],
        temperature=0.3,
        max_tokens=700
    )
    return response.choices[0].message.content.strip()

# ✅ 검증 함수
def verify_pair(problem, question, reject, chosen, correct_code):
    prompt = f"""
문제:
{problem}

질문:
{question}

1. reject:
{reject}

2. chosen:
{chosen}

검증 기준:
- reject는 실제 오류와 무관한 엉뚱한 이유여야 합니다.
- chosen은 주어진 정답 코드에 기반해야 합니다.
- 다른 방식(while문 등)을 쓰면 FAIL입니다.
- 기준에 맞으면 PASS, 아니면 FAIL만 출력하세요.
"""
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 reject/chosen 검증 AI야. 'PASS' 또는 'FAIL'만 출력해."},
            {"role": "user", "content": prompt.strip()}
        ],
        temperature=0.0,
        max_tokens=10
    )
    return response.choices[0].message.content.strip()

# ✅ 결과 저장 리스트
dpo_dataset = []
invalid_logs = []

# ✅ 생성 시작
for ex in tqdm(examples, desc="DPO 샘플 생성 중"):
    question = ex["question"]

    reject = generate_incorrect(ex["problem"], ex["buggy_code"], question)
    chosen = generate_correct(
        ex["problem"], question, ex["buggy_code"],
        ex["error_reason"], ex["correct_code"], reject
    )
    verified = verify_pair(ex["problem"], question, reject, chosen, ex["correct_code"])

    if verified.strip() == "PASS":
        dpo_dataset.append({
            "prompt": f"{ex['problem']}\n\n{question}\n\n코드:\n{ex['buggy_code']}",
            "rejected": reject,
            "chosen": chosen
        })
    else:
        invalid_logs.append({
            "problem": ex["problem"],
            "question": question,
            "reject": reject,
            "chosen": chosen
        })

# ✅ 저장
with open("dpo_dataset_verified.json", "w", encoding="utf-8") as f:
    json.dump(dpo_dataset, f, indent=2, ensure_ascii=False)

with open("dpo_invalid_log.json", "w", encoding="utf-8") as f:
    json.dump(invalid_logs, f, indent=2, ensure_ascii=False)

print(f"✅ 최종 저장된 DPO 샘플 수: {len(dpo_dataset)}개")
print(f"🚫 검증 실패로 로그에 저장된 샘플 수: {len(invalid_logs)}개")


DPO 샘플 생성 중: 100%|██████████| 100/100 [20:25<00:00, 12.25s/it]

✅ 최종 저장된 DPO 샘플 수: 98개
🚫 검증 실패로 로그에 저장된 샘플 수: 2개
